### Computer use with agents

This notebook will show you how to to create an agent that can use the computer.

Currently only Claude 3.7 Sonnet has been tested with this notebook. It can currently
only perform click, type, and scroll.

**Prerequisites**:

- Access to Amazon Bedrock Claude 3.7 Sonnet


In [ ]:
from time import sleep

from converseagent.messages import UserMessage
from converseagent.models.bedrock import BedrockModel
from converseagent.models.config import InferenceConfig
from converseagent_extras.agents.computer_agent import ComputerAgent
from converseagent.utils.update import update_callback

In [ ]:
# Specify a Bedrock model id
bedrock_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

model = BedrockModel(
    bedrock_model_id=bedrock_model_id,
)

# Setting inference config
inference_config = InferenceConfig(max_tokens=32000, temperature=1)

# Enabling Claude 3.7 Sonnet Reasoning
additional_model_request_fields = {
    "thinking": {"type": "enabled", "budget_tokens": 1024},
    "anthropic_beta": ["computer-use-2025-01-24"],
}

agent = ComputerAgent(model=model)

In [ ]:
prompt = """
Try to be efficient in your computer use. If you can combine both move and click in one go, then
make sure to use the click tool with the coordinates. Use the red dot to tell you where

Go to amazon.ca, find a Fire TV 4K at least 50" big, and add it to the cart.
Decline any extended warranty.
"""

user_message = UserMessage(text=prompt)

# Give time to bring up the web browser
sleep(5)

# Initial screenshot to get the
initial_screenshot = agent._get_screenshot()
initial_screenshot.metadata = {"retention": "after_next_turn"}
user_message.append_content(initial_screenshot)

response = agent.invoke(
    user_message=user_message,
    inference_config=inference_config,
    additional_model_request_fields=additional_model_request_fields,
    verbose=False,
    max_iterations=50,
    update_callback=update_callback,
)
print(response["body"]["text"])